In [14]:
import numpy as np
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import math
import threading

In [15]:
np.__config__.show()

blas_armpl_info:
  NOT AVAILABLE
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/tycam/anaconda3/envs/gpu/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/tycam/anaconda3/envs/gpu/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/tycam/anaconda3/envs/gpu/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/tycam/anaconda3/envs/gpu/include']
lapack_armpl_info:
  NOT AVAILABLE
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/tycam/anaconda3/envs/gpu/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/tycam/anaconda3/envs/gpu/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/tycam/anaconda3/envs/gpu/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/tycam/a

In [16]:

OPENBLAS_NUM_THREADS

NameError: name 'OPENBLAS_NUM_THREADS' is not defined

In [17]:
class DFGF:
	#parameters for the simulation
	s = 0.0
	n = 0
	numTrials = 0
	isDirichlet  = True
	def __init__(self):
		pass
	#eigenvalues and eigenvectors for the discrete fractional gaussian field
	#their specific sizes will change depening on the degree of the approximation 
	#and the dimension that is being approximated.
	eigenValues = np.array((n,1))
	eigenVectors = np.array((n,n))

	#coefficients are calculated using the eigenvectors and eigenvalues
	coefficients = None

	#ndarray of iid standard normals
	rng = np.random.default_rng(1020304050)
	sample = None

	#data resulting from DFGF calculations
	trialData = {}
	maximaVector = None
	meanOfMaxima = 0





	########################################
	#           Getters & Setters          #
	########################################

	def getTrialData(self):
		return self.trialData
	def getMaximaVector(self):
		return self.maximaVector
	def getMeanOfMaxima(self):
		return self.meanOfMaxima
	def getCoefficients(self):
		return self.coefficients
	def getEigenValues(self):
		return self.eigenValues
	def getEigenVectors(self):
		return self.eigenVectors
	def setSample(self, sample):
		self.sample = sample
	def setCoefficients(self, coeffs):
		self.ceofficients = coeffs
	def setEigenValues(self, vals):
		self.eigenValues = vals

	def reuseArrays(self, eigenValues, eigenVectors, sample):
		self. eigenValues = setEigenValues
		self.eigenVectors = getEigenVectors
		self.sample = sample


In [18]:
class DFGF_S1(DFGF):
	
	def __init__(self, sRange, n, numTrials, isDirchlet, compute):
		#set parameters
		self.sRange = sRange
		self.n=n
		self.numTrials = numTrials
		self.isDirchlet = isDirchlet

		if compute:
			self.computeSample()
			print("sample computed")
			self.computeEigenValues()
			print("eigenvalues computed")
			self.computeEigenVectors()
			print("eigenvectors computed")
			self.computeCoefficients(self.s)
			print("coefficients computed")


	def computeSample(self):
		#could use an nxnxnumTrials matrix and matrix multiplication but this could take up huge amounts of memory...right?
		self.sample = self.rng.standard_normal((self.numTrials, self.n))

	def computeEigenValues(self):
		tempVector = np.arange(1, math.ceil(self.n/2)+1)
		self.eigenValues = self.n**2/(2*np.pi**2)*(1-np.cos(2*np.pi*(tempVector)/self.n))
		tempVector = np.arange(1, math.floor(self.n/2)+1)
		self.eigenValues = np.append(self.eigenValues, self.n**2/(2*np.pi**2)*(1-np.cos(2*np.pi*(tempVector)/self.n)))

	def computeEigenVector(self, k):
		if self.isDirchlet == False:
			tempEigenVectorSines = np.arange(1,math.floor(self.n/2)+1)
			tempEigenVectorCosines = np.arange(1, math.ceil(self.n/2)+1)
			sines = (np.sin(2*np.pi*k/self.n * tempEigenVectorSines))
			cosines = np.cosine(2*np.pi*k/self.n * tempEigenVectorCosines)
			return np.append(consines,sines)
		elif self.isDirchlet == True:
			tempEigenVectorSines = np.arange(1,math.floor(self.n/2)+1)
			sines = (np.sin(2*np.pi*k/self.n * tempEigenVectorSines))
			cosines = np.zeros((math.ceil(self.n/2)))
			return np.append(cosines, sines).reshape(1,self.n)
	def computeEigenVectors(self):
		self.eigenVectors = self.computeEigenVector(0)
		for k in np.arange(1,self.n):
			self.eigenVectors = np.insert(self.eigenVectors,self.eigenVectors.shape[0], self.computeEigenVector(k),axis  = 0)

	def computeCoefficients(self,s):
		denominators = np.power(self.eigenValues, -s)
		self.coefficients = np.multiply(self.eigenVectors , denominators)

	def evaluate(self,trialNum):
		return np.dot(self.coefficients, self.sample[trialNum])

	def computeTrial(self, trialNum):
		print(trialNum)
		self.trialData[trialNum] = self.evaluate(trialNum)
		print(self.evaluate(trialNum))

	def runTrials(self):
# 		# python multiprocessing
# 		print("computing tirals")
# 		num_workers = mp.cpu_count()
# 		tasks = [*range(self.numTrials)] 
# 		pool = mp.Pool(num_workers)
# 		for task in tasks:
# 			pool.apply_async(self.computeTrial, args = (int(task), self.trialData))

# 		pool.close()
# 		pool.join()
		# print(num_workers)
		# pool = mp.Pool(num_workers)
		# pool.map(self.computeTrial, [1,2,3,4])
		# pool.close()
		# pool.join()

		# python threading
		threads = []
		for trialNum in np.arange(self.numTrials):
			thread = threading.Thread(target = self.computeTrial, args =(trialNum,))
			thread.start()
			threads.append(thread)
		for thread in threads:
			thread.join()

		# no threading
		# for trialNum in np.arange(self.numTrials):
		# 	self.computeTrial(trialNum)



dfgf = DFGF_S1(.05, 5000, 2000, True, True)

dfgf.runTrials()
print(len(dfgf.getTrialData()))


sample computed
eigenvalues computed
eigenvectors computed
coefficients computed
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
[  0.          10.2410582    2.42113788 ...  23.7088071   -2.42113788
 -10.2410582 ]
[  0.          -4.30601832 -11.19566566 ... -30.16890433  11.19566566
   4.30601832]
42
43
[  0.         -42.24952088  54.66579136 ...  62.20059449 -54.66579136
  42.24952088]
[  0.         -13.48651064 -33.00112348 ... -33.87788173  33.00112348
  13.48651064]
[  0.          11.3285416  -26.62799063 ...  -0.58174924  26.62799063
 -11.3285416 ][  0.         -15.92365182  25.94062183 ... -64.20088859 -25.94062183
  15.92365182]
44[  0.         -17.22149459   2.29400068 ...  -0.62385915  -2.29400068
  17.22149459]
[  0.          48.35943824 -63.6854063  ...  22.06663507  63.6854063
 -48.35943824]
[  0.         -45.61014049 -57.86664098 ... -24.60395497  57.86664098
  45.61014049]

[  0.          15.33152259 -22.